# Institulac 

In [33]:
import re
import json
import time
from datetime import datetime

import pandas as pd
pd.set_option("max_rows",100)
#pd.set_option("display.max_columns",100)
pd.set_option("max_colwidth",1000)

import helium as h
from selenium.common.exceptions import NoSuchElementException

In [34]:
# DICT CAT-PRODS-TAB
with open('dict_tables.json') as file_json:
    dict_tables=json.loads(file_json.read())

In [35]:
# login =
# name_ins =
# usser =
# passw=

# login
browser = h.start_firefox('https://scienti.minciencias.gov.co/institulac2-war/')
sleep=0.8

#browser = h.start_firefox('https://scienti.minciencias.gov.co/institulac2-war/')
time.sleep(sleep)
h.click('Consulte Aquí')

time.sleep(sleep)
h.write('UNIVERSIDAD DE ANTIOQUIA',into='Digite el nombre de la Institución') # name ins

time.sleep(sleep)
h.click('Buscar')

time.sleep(sleep)
h.click(browser.find_element_by_id('list_instituciones'))

time.sleep(sleep)

time.sleep(sleep)
h.select('seleccione una','UNIVERSIDAD DE ANTIOQUIA') # name_ins

time.sleep(sleep)
h.write('annyarango',into='Usuario')                  # user

time.sleep(sleep)
h.write('1@Silver', into='Contraseña')                # passw

time.sleep(sleep)
h.click(h.Button('Ingresar'))

# cookie injection
time.sleep(sleep)
# implementation cookie injection

# get current cookie and store
new_cookie=browser.get_cookies()[0]
    
# create new_cookie with time_expire
time_expire = (datetime(2022,1,1) - datetime(1970,1,1)).total_seconds()
new_cookie['expiry'] = int(time_expire)
    
# delete cookie sites
browser.delete_all_cookies()
    
# add new cookie
browser.add_cookie(new_cookie)

# navigation 1
time.sleep(sleep)
h.click('Aval')

time.sleep(sleep)
h.click('Avalar grupos')

time.sleep(sleep)
h.click('Grupos Avalados')

# -- end login --

# list of total groups
#select max results per page
h.wait_until(h.Text('Ver Reporte').exists)
h.click(browser.find_element_by_xpath('//table[@id="grupos_avalados"]//select[@name="maxRows"]'))

time.sleep(sleep)
h.select(browser.find_element_by_xpath('//table[@id="grupos_avalados"]//select[@name="maxRows"]'),'100')

# catch 1: groups info [name, lider, cod,  link to producs]  
# schema
# empty df
# select max items per page
# while until end
# try:
    # catch table
    # preproces table
    # catch urls
    # add url colums
    # add df
    # click next page -> raise error
# except Nosuchelement:
    # break
    
# catch 1: list of groups
dfg=pd.DataFrame()
cont=True

while cont:
    
    try:
        # catch source
        time.sleep(sleep)
        source_g=browser.page_source
        
        # catch table
        time.sleep(sleep)
        df=pd.read_html(source_g, attrs={"id":"grupos_avalados"}, header=2)[0]
        
        # and preprocces it
        c=[x for x in df.columns if x.find('Unnamed:') == -1]
        dfgp=df[c][1:-1]
        print(dfgp.columns,dfgp.shape)
        
        # catch urls
        url=[a.get_attribute('href') for a in browser.find_elements_by_xpath('//table[@id="grupos_avalados"]//td[5]/a')]
        dfgp['Revisar'] = url
        dfg=dfg.append(dfgp)
        
        # click next page. this instruction rise error of the end. 
        h.click(browser.find_element_by_xpath('//table[@id="grupos_avalados"]//tr/td[3]/a'))
        
    except NoSuchElementException as e:
        
        print(e)
        print('out of cicle')
        break
        
    time.sleep(sleep)
    time.sleep(sleep)
    
dfg = dfg.reset_index(drop=True)
assert dfg.shape[0] == 324

Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (100, 4)
Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (100, 4)
Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (100, 4)
Index(['Nombre del grupo', 'Nombre del líder', 'COL Grupo', 'Revisar'], dtype='object') (24, 4)
Message: Unable to locate element: //table[@id="grupos_avalados"]//tr/td[3]/a

out of cicle


In [12]:
print(dfg['Nombre del grupo'][0:1].values[0],'\n',dfg['Revisar'][:1].values[0])

print(dfg['Nombre del grupo'][-1:].values[0],'\n',dfg['Revisar'][-1:].values[0])

Grupo Tandem en Nano-bio-física 
 https://scienti.minciencias.gov.co/institulac2-war/ReGrupoInstitucion/query.do?avalGr=T&codigoGrupo=&idGrupo=00000000020456
Promoción de la Salud 
 https://scienti.minciencias.gov.co/institulac2-war/ReGrupoInstitucion/query.do?avalGr=T&codigoGrupo=&idGrupo=00000000001767


In [10]:
dfg['Nombre del grupo'][3:4]

3    Micología Médica
Name: Nombre del grupo, dtype: object

In [51]:
time.sleep(sleep*2)

DB = [] # 
LP = []
LR = [] 
for idx in dfg.index[3:4]:
    
    # create db for store things related to group
    DBG = {}
    
    # part info group
    print(dfg.loc[idx,'Nombre del grupo'])

    # specific group url
    time.sleep(sleep)
    url_group = dfg.loc[idx,'Revisar']

    # go to url group
    time.sleep(sleep)
    browser.get(url_group)

    # catch two tables: info grupo and  members
    source=browser.page_source

    #info
    l_info=pd.read_html(source, match='Nombre Grupo')
    info_g=l_info[3].pivot(columns=0,values=1)
    
    # STORE INFO_GROUP
    DBG['Info_group'] = info_g

    # members
    l_int = pd.read_html(source,attrs={'id':'tblIntegrantes'},header=2)
    mem_g=l_int[0]
    
    # STORE_MEMBERS
    DBG['Members'] =  mem_g

    # Products

    #time.sleep(sleep*5) # time time time !!!
    h.wait_until(lambda: browser.find_element_by_xpath('//td[@id="bodyPrincipal"]//a[text()="Ver productos"]') is not None)
    h.click(browser.find_element_by_xpath('//td[@id="bodyPrincipal"]//a[text()="Ver productos"]'))

    # products by belongs to  # time time time
    #time.sleep(sleep*7)       # time time time
    h.wait_until(lambda: browser.find_element_by_xpath('//*[@id="ProdsPertenecia"]') is not None)
    h.click(browser.find_element_by_xpath('//*[@id="ProdsPertenecia"]'))

    time.sleep(sleep)
    url_products=browser.current_url


    # map all products, store those id categories that amount is different to 0 and id products asociated.
    # make queries with combinations of categories and products
    # make urls with diferent combinations of quieries
    # go to each of urls
    # load page source
    # catch table ( or tables) asociated with categories and products
    # store tables

    report = ''

    list_of_prods =[] #[[cat,prod],[cat,prod]...]

    # map all products and get products and subs diff to cero
    for i in browser.find_elements_by_xpath('//div[@id="accordionCatgP"]/h3'):

        report += i.text + '\n' 
        report += i.get_attribute('id') + '\n'     

        time.sleep(sleep)
        h.click(i)
        
        # cat
        cat_ = int(re.findall(r'\d+',i.text)[0])
        
        # create cat key in dict, for estore diferents products by this categori: 'NC_': {'ART_E':TABLE,
        #                                                                                 'ART_IMP':TABLE}
        #if cat_ > 0:
        DBG[i.get_attribute('id')] = {}
            
        
        for j in browser.find_elements_by_xpath('//div[@aria-labelledby="%s"]/h3' % i.get_attribute('id')):

            report += '\t' + j.text + '\n' 
            report += '\t' + j.get_attribute('id') + '\n'
            
            #prod
            pro_ = int(re.findall(r'\d+', j.text)[0])

            #if cat_ > 0 and pro_ > 0:  
                
            list_of_prods.append([i.get_attribute('id'),j.get_attribute('id')])

        time.sleep(sleep) 
        # h.click(a)
        h.click(i)
    
    # PAR: products with revisions
    h.wait_until(lambda: browser.find_element_by_xpath('//*[@id="ProdsAval"]'))
    h.click(browser.find_element_by_xpath('//*[@id="ProdsAval"]'))

    # NC

    _NC = browser.find_element_by_xpath('//*[@id="NC"]')

    h.click(_NC)

    cat_ = int(re.findall(r'\d+',_NC.text)[0])

    LIB = browser.find_element_by_xpath('//*[@id="LIB"]')

    L = int(re.findall(r'\d+', LIB.text)[0])

    CAP_LIB = browser.find_element_by_xpath('//*[@id="CAP_LIB"]')

    CL = int(re.findall(r'\d+', CAP_LIB.text)[0])

    #if (cat_ > 0 and L > 0) or (cat_ > 0 and CL > 0):

    DBG[_NC.get_attribute('id')] = {}
        
    #if (cat_ > 0 and L > 0):
        
    list_of_prods.append([_NC.get_attribute('id'),LIB.get_attribute('id')])
    
    #if (cat_ > 0 and CL > 0):
        
    list_of_prods.append([_NC.get_attribute('id'),CAP_LIB.get_attribute('id')])
                
    # print(report)
    # print('\n')
    # print('--------------------------------')
    time.sleep(sleep*2)
    
    tables=[]
    
    for p in range(len(list_of_prods)):

            # make query
            if list_of_prods[p][0] == 'NC':
                
                query='categoria=%s&subcategoria=%s&aval=T' % (list_of_prods[p][0],list_of_prods[p][1])
                
            else:
                
                query='categoriaP=%s&subcategoriaP=%s&aval=P' % (list_of_prods[p][0],list_of_prods[p][1])

            # make url query
            url_query = url_products.split('?')[0] + '?' + query + '&' + url_products.split('?')[1]

            # retrieve id asociated tables
            table_id = dict_tables[list_of_prods[p][0]][list_of_prods[p][1]]

            # go to url product by group
            time.sleep(sleep)
        
            browser.get(url_query)

            # load page
            time.sleep(sleep)
            page_source = browser.page_source

            # catch tables
            if isinstance(table_id,str): # case one table

                # catch title table
                
                #title_table = browser.find_element_by_xpath('//div/p[@class="titulo_tabla"]').text 
                # cathc table
                print(url_query)
                time.sleep(sleep*2)
                table = pd.read_html(page_source,attrs={'id':table_id}, header=2)[0][1:-1]

                # store table
                DBG[list_of_prods[p][0]][list_of_prods[p][1]] = {table_id:table}
                # ---- in building ----

            elif isinstance(table_id, list): # case multiple tables
                
                DBG[list_of_prods[p][0]][list_of_prods[p][1]] ={}

                for i in range(len(table_id)):
                    
                    # fix bug
                    if list_of_prods[p][1] == 'DC_P' and i == 3:
                        # catch title specific table 
                        title_table = browser.find_elements_by_xpath('//div/p[@class="titulo_tabla"]')[i].text
                        
                        # catch table software
                        table = pd.read_html(page_source,attrs={'id':table_id[i]}, header=2)[1][1:-1]
                        
                        # store table
                        DBG[list_of_prods[p][0]][list_of_prods[p][1]]['DC_DES_P_TABLE'] = table
                        

                    # catch title specific table 
                    title_table = browser.find_elements_by_xpath('//div/p[@class="titulo_tabla"]')[i].text

                    # catch table trasmedia
                    table = pd.read_html(page_source,attrs={'id':table_id[i]}, header=2)[0][1:-1]

                    # store table
                    DBG[list_of_prods[p][0]][list_of_prods[p][1]][table_id[i]]=table
                
        
                    # -----------
    DB.append(DBG)
    LP.append(list_of_prods)
    LR.append(report)

Micología Médica
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=NC_P&subcategoriaP=ART_IMP_P&aval=P&idGrupo=00000000002499&avalGr=T&codigoGrupo=&grCAv=1&nmeGrupo=Micolog%EDa%20M%E9dica&codIdGrupo=COL0042069
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=NC_P&subcategoriaP=ART_ELE_P&aval=P&idGrupo=00000000002499&avalGr=T&codigoGrupo=&grCAv=1&nmeGrupo=Micolog%EDa%20M%E9dica&codIdGrupo=COL0042069
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=NC_P&subcategoriaP=LIB_P&aval=P&idGrupo=00000000002499&avalGr=T&codigoGrupo=&grCAv=1&nmeGrupo=Micolog%EDa%20M%E9dica&codIdGrupo=COL0042069
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=NC_P&subcategoriaP=CAP_LIB_P&aval=P&idGrupo=00000000002499&avalGr=T&codigoGrupo=&grCAv=1&nmeGrupo=Micolog%EDa%20M%E9dica&codIdGrupo=COL0042069
https://scienti.minciencias.gov.co/institulac2-

https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=ASC_P&subcategoriaP=BOL_RES_INV_P&aval=P&idGrupo=00000000002499&avalGr=T&codigoGrupo=&grCAv=1&nmeGrupo=Micolog%EDa%20M%E9dica&codIdGrupo=COL0042069
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=ASC_P&subcategoriaP=CON_INF_TEC_P&aval=P&idGrupo=00000000002499&avalGr=T&codigoGrupo=&grCAv=1&nmeGrupo=Micolog%EDa%20M%E9dica&codIdGrupo=COL0042069
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=ASC_P&subcategoriaP=EDIC_P&aval=P&idGrupo=00000000002499&avalGr=T&codigoGrupo=&grCAv=1&nmeGrupo=Micolog%EDa%20M%E9dica&codIdGrupo=COL0042069
https://scienti.minciencias.gov.co/institulac2-war/verificador/allproductosGr.do?categoriaP=FRH_P&subcategoriaP=TES_DOC_P&aval=P&idGrupo=00000000002499&avalGr=T&codigoGrupo=&grCAv=1&nmeGrupo=Micolog%EDa%20M%E9dica&codIdGrupo=COL0042069
https://scienti.minciencias.gov.co/institulac2-war/

In [52]:
list_of_prods

[['NC_P', 'ART_IMP_P'],
 ['NC_P', 'ART_ELE_P'],
 ['NC_P', 'LIB_P'],
 ['NC_P', 'CAP_LIB_P'],
 ['NC_P', 'NOT_CIE_P'],
 ['NC_P', 'PAT_P'],
 ['NC_P', 'PRD_INV_ART_P'],
 ['NC_P', 'VAR_VEG_P'],
 ['NC_P', 'VAR_ANI_P'],
 ['NC_P', 'RAZ_PEC_P'],
 ['NC_P', 'TRA_FIL_P'],
 ['DTI_P', 'DIS_IND_P'],
 ['DTI_P', 'CIR_INT_P'],
 ['DTI_P', 'SOFT_P'],
 ['DTI_P', 'NUTRA_P'],
 ['DTI_P', 'COL_CIENT_P'],
 ['DTI_P', 'REG_CIENT_P'],
 ['DTI_P', 'PLT_PIL_P'],
 ['DTI_P', 'PRT_IND_P'],
 ['DTI_P', 'SEC_IND_P'],
 ['DTI_P', 'PROT_VIG_EPID_P'],
 ['DTI_P', 'EMP_BSE_TEC_P'],
 ['DTI_P', 'EMP_CRE_CUL_P'],
 ['DTI_P', 'INN_GES_EMP_P'],
 ['DTI_P', 'INN_PROC_P'],
 ['DTI_P', 'REG_NORM_REGL_LEG_P'],
 ['DTI_P', 'CONP_TEC_P'],
 ['DTI_P', 'REG_AAD_P'],
 ['DTI_P', 'SIG_DIS_P'],
 ['ASC_P', 'GEN_CONT_IMP_P'],
 ['ASC_P', 'EV_CIENT_P'],
 ['ASC_P', 'RED_P'],
 ['ASC_P', 'TALL_CRE_P'],
 ['ASC_P', 'EVE_ART_P'],
 ['ASC_P', 'DOC_TRAB_P'],
 ['ASC_P', 'SEC_GENE_P'],
 ['ASC_P', 'BOL_RES_INV_P'],
 ['ASC_P', 'CON_INF_TEC_P'],
 ['ASC_P', 'EDIC_P'],
 

In [107]:
DB[0]

{'Info_group': 0  NaN  (Fecha de última actualización del aval  10 de junio de 2019)  \
 0  NaN  (Fecha de última actualización del aval  10 de junio de 2019)   
 1  NaN                                                            NaN   
 2  NaN                                                            NaN   
 3  NaN                                                            NaN   
 4  NaN                                                            NaN   
 5  NaN                                                            NaN   
 6  NaN                                                            NaN   
 
 0      Nombre Grupo              Nombre Líder       Nombre área  \
 0               NaN                       NaN               NaN   
 1               NaN                       NaN               NaN   
 2  Micología Médica                       NaN               NaN   
 3               NaN                       NaN               NaN   
 4               NaN                       NaN  Medi

In [57]:
DB[0]['ASC_P']['GEN_CONT_IMP_P']

{'GC_I_P_TABLE_1': Empty DataFrame
 Columns: [Unnamed: 0, Tipo de producto, Nombre revista, Título del artículo, Año de presentación, Volumen, Página inicial, Página final, ISSN, Medio divulgación-URL, DOI, Categoría, Última actualización, Revisar]
 Index: [],
 'GC_I_P_TABLE_5': Empty DataFrame
 Columns: [Unnamed: 0, Título del libro, ISBN, Editorial, Lugar de publicación, Categoría, Fecha de publicación, Revisar, Unnamed: 8]
 Index: [],
 'GC_I_P_TABLE_6': Empty DataFrame
 Columns: [Unnamed: 0, Título del libro, ISBN, Editorial, Lugar de publicación, Categoría, Fecha de publicación, Proyecto de investigación, Revisar]
 Index: [],
 'GC_I_P_TABLE_7': Empty DataFrame
 Columns: [Unnamed: 0, Título del libro, ISBN, Editorial, Lugar de publicación, Categoría, Fecha de publicación, Proyecto de investigación, Revisar]
 Index: []}

In [58]:
DB[0]['ASC_P']['DC_P']

{'DC_CD_P_TABLE': Empty DataFrame
 Columns: [Título del proyecto, Autor/Coautores, Institución, Lugar, Fecha realización del proyecto, Fecha fin realización del proyecto, Medio de verificación, Revisar, Unnamed: 8]
 Index: [],
 'DC_CON_P_TABLE': Empty DataFrame
 Columns: [Unnamed: 0, Título del proyecto, Autor/Coautores, Institución, Lugar, Fecha inicio realización del proyecto, Fecha fin realización del proyecto, Categoría, Medio de verificación, Revisar]
 Index: [],
 'DC_TRA_P_TABLE': Empty DataFrame
 Columns: [Unnamed: 0, Título del proyecto, Autor/Coautores, Institución, Lugar, Fecha realización del proyecto, Fecha fin realización del proyecto, Categoría, Medio de verificación, Revisar]
 Index: [],
 'DC_DES_P_TABLE': Empty DataFrame
 Columns: [Unnamed: 0, Título del proyecto, Autor/Coautores, Institución, Lugar, Fecha realización del proyecto, Fecha fin realización del proyecto, Categoría, Medio de verificación, Revisar]
 Index: []}

In [59]:
list_of_prods

[['NC_P', 'ART_IMP_P'],
 ['NC_P', 'ART_ELE_P'],
 ['NC_P', 'LIB_P'],
 ['NC_P', 'CAP_LIB_P'],
 ['NC_P', 'NOT_CIE_P'],
 ['NC_P', 'PAT_P'],
 ['NC_P', 'PRD_INV_ART_P'],
 ['NC_P', 'VAR_VEG_P'],
 ['NC_P', 'VAR_ANI_P'],
 ['NC_P', 'RAZ_PEC_P'],
 ['NC_P', 'TRA_FIL_P'],
 ['DTI_P', 'DIS_IND_P'],
 ['DTI_P', 'CIR_INT_P'],
 ['DTI_P', 'SOFT_P'],
 ['DTI_P', 'NUTRA_P'],
 ['DTI_P', 'COL_CIENT_P'],
 ['DTI_P', 'REG_CIENT_P'],
 ['DTI_P', 'PLT_PIL_P'],
 ['DTI_P', 'PRT_IND_P'],
 ['DTI_P', 'SEC_IND_P'],
 ['DTI_P', 'PROT_VIG_EPID_P'],
 ['DTI_P', 'EMP_BSE_TEC_P'],
 ['DTI_P', 'EMP_CRE_CUL_P'],
 ['DTI_P', 'INN_GES_EMP_P'],
 ['DTI_P', 'INN_PROC_P'],
 ['DTI_P', 'REG_NORM_REGL_LEG_P'],
 ['DTI_P', 'CONP_TEC_P'],
 ['DTI_P', 'REG_AAD_P'],
 ['DTI_P', 'SIG_DIS_P'],
 ['ASC_P', 'GEN_CONT_IMP_P'],
 ['ASC_P', 'EV_CIENT_P'],
 ['ASC_P', 'RED_P'],
 ['ASC_P', 'TALL_CRE_P'],
 ['ASC_P', 'EVE_ART_P'],
 ['ASC_P', 'DOC_TRAB_P'],
 ['ASC_P', 'SEC_GENE_P'],
 ['ASC_P', 'BOL_RES_INV_P'],
 ['ASC_P', 'CON_INF_TEC_P'],
 ['ASC_P', 'EDIC_P'],
 

In [61]:
print(LR[0])

Nuevo Conocimiento (2)
NC_P
	Artículos publicados en revistas especializadas - Impresos (1)
	ART_IMP_P
	Artículos publicados en revistas especializadas - Electrónicos (0)
	ART_ELE_P
	Libros resultado de investigación (0)
	LIB_P
	Capítulos en Libro resultado de investigación (1)
	CAP_LIB_P
	Notas científicas (0)
	NOT_CIE_P
	Patentes (0)
	PAT_P
	Obras o productos de investigación creación en Artes, Arquitectura y Diseño (0)
	PRD_INV_ART_P
	Variedad Vegetal (0)
	VAR_VEG_P
	Nueva Raza Animal (0)
	VAR_ANI_P
	Poblaciones mejoradas de razas pecuarias (0)
	RAZ_PEC_P
	Traducciones Filológicas y Edición de Fuentes (0)
	TRA_FIL_P
Desarrollo Tecnológico e Innovación (0)
DTI_P
	Diseño industrial (0)
	DIS_IND_P
	Circuitos integrados (0)
	CIR_INT_P
	Software (0)
	SOFT_P
	Productos nutraceúticos (0)
	NUTRA_P
	Colecciones científicas (0)
	COL_CIENT_P
	Nuevos registros científicos (0)
	REG_CIENT_P
	Planta Piloto (0)
	PLT_PIL_P
	Prototipo Industrial (0)
	PRT_IND_P
	Secreto Industrial (0)
	SEC_IND_P
	Prot

In [65]:
assert len(LD)==len(LP)

In [66]:
def get_info(df):
    
    info= {
        'Nombre_Grupo' : df['Nombre Grupo'].dropna().iloc[0],

        'Nombre_Lider' : df['Nombre Líder'].dropna().iloc[0],

        'Nombre_Area'  : df['Nombre área'].dropna().iloc[0]
    }
    
    dfi = pd.DataFrame(info, index=[0])
  
    
    return dfi

In [67]:
DBEH = {
    
    'INFO_GROUP': 'TABLE',
    'MEMBERS':['Identificación Nacionalidad','Tiene afiliación con UdeA', 'Si no tiene afiliación UdeA diligencie el nombre de la Institución','Nro. Horas de dedicación semanales que avala el Coordinador de grupo'], # 2
       
    'NC_P': {'ART_IMP_P': {'ART_P_TABLE':['URL','DOI','Si no tiene URL o DOI agregue una evidencia en el repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'ART_ELE_P': {'ART_E_P_TABLE':['URL','DOI','Si no tiene URL o DOI agregue una evidencia en el repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'LIB_P':     {'LIB_P_TABLE':['Proyecto de investigación del cual se derivó el libro (Código-Título)','Financiador(es) del proyecto del cual se derivó el libro', 'Financiador(es) de la publicación','Autores','Citas recibidas (si tiene)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'CAP_LIB_P': {'CAP_LIB_P_TABLE':['Proyecto de investigación del cual se derivó el libro que contiene el capítulo (Código-Título)','Financiador del proyecto del cual se derivó el libro que contiene el capítulo','Financiador de la publicación','Autores','Citas recibidas (si tiene)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'NOT_CIE_P': {'NOT_CIE_P_TABLE':['URL','DOI','Si no tiene URL o DOI genere una evidencia en el repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'PAT_P':     {'PAT_P_TABLE':['Autores', 'Examen de fondo favorable','Examen preliminar internacional favorable','Adjunta opiniones escritas de la bUsqueda internacional','Contrato de explotación','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}, #  123 -1
             'PRD_INV_ART_P': {'PAAD_P_TABLE':['Autores','Tiene certificado institucional de la obra','Tiene certificado de la entidad que convoca al evento en el que participa','Tiene certificado de la entidad que convoca al premio en el que obtiene','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}, # 1 2 3 -1
             'VAR_VEG_P':     {'VV_P_TABLE':['Autores','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'VAR_ANI_P':     {'VA_P_TABLE':['Autores','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'RAZ_PEC_P':     {'RAZ_PEC_P_TABLE':['Autores','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
             'TRA_FIL_P': {'TRA_FIL_P_TABLE':['Proyecto de investigación del cual se derivó el libro (Código-Título)','Financiador(es) del proyecto del cual se derivó el libro','Financiador(es) de la publicación','Autores','Citas recibidas (si tiene)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}
            },
     'DTI_P': {'DIS_IND_P': {'DI_P_TABLE':['Autores','Contrato (si aplica)','Nombre comercial (si aplica)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'CIR_INT_P': {'ECI_P_TABLE':['Autores','Contrato (si aplica)','Nombre comercial (si aplica)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'SOFT_P': {'SF_P_TABLE':['Autores','Contrato (si aplica)','Nombre comercial (si aplica)','TRL','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'NUTRA_P': {'NUTRA_P_TABLE':['Autores','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']}, # add
              'COL_CIENT_P': {'COL_CIENT_P_TABLE':['Autores','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo', '¿El producto cumple con los requisitos para ser avalado?']},
              'REG_CIENT_P': {'REG_CIENT_P_TABLE':['Autores','Contrato licenciamiento (si aplica)','Agregue las evidencias verificadas al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'PLT_PIL_P': {'PP_P_TABLE':['Autores','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'PRT_IND_P': {'PI_P_TABLE':['Autores','Nombre comercial (si aplica)','TRL','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'SEC_IND_P': {'SE_P_TABLE':['Autores','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'PROT_VIG_EPID_P': {'PROT_VIG_EPID_P_TABLE':['Autores','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'EMP_BSE_TEC_P': {'EBT_P_TABLE':['Autores','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'EMP_CRE_CUL_P': {'ICC_P_TABLE':['Autores','Agregue la evidencia verificada al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'INN_GES_EMP_P': {'IG_P_TABLE':['Autores','Contrato (si aplica)','Nombre comercial (si aplica)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'INN_PROC_P': {'IPP_P_TABLE':['Autores','Contrato (si aplica)','Nombre comercial (si aplica)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'REG_NORM_REGL_LEG_P': {'RNR_P_TABLE':['Autores','Contrato (si aplica)','Convenio (si aplica)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'CONP_TEC_P': {'CONP_TEC_P_TABLE':['Autores','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'REG_AAD_P': {'AAAD_P_TABLE':['Autores','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'SIG_DIS_P': {'SD_P_TABLE':['Autores','Contrato licenciamiento (si aplica)','Agregue las evidencias verificadas al repositorio digital y copie el link del archivo en este campo','¿El producto cumple con los requisitos para ser avalado?']}
              },
    'ASC_P': {'GEN_CONT_IMP_P': {'GC_I_P_TABLE_5':['Autores','Citas recibidas (si tiene)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'PASC_P': {'PASC_FOR_P_TABLE':['Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?'],
               'PASC_TRA_P_TABLE':['Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?'],
               'PASC_GEN_P_TABLE':['Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?'],
               'PASC_CAD_P_TABLE':['Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'DC_P': {'DC_CD_P_TABLE':['Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?'],
               'DC_CON_P_TABLE':['Medio de verificación','Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?'],
               'DC_TRA_P_TABLE':['Medio de verificación','Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?'],
               'DC_DES_P_TABLE':['Medio de verificación','Proyecto/Código','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}},
    
    'FRH_P': {'TES_DOC_P': {'TD_P_TABLE':['Número de cédula del graduado','¿La fecha fin coincide con la fecha de grado del estudiante?','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},  # 1 -1
              'TES_MAST_P': {'TM_P_TABLE':['Número de cédula del graduado','¿La fecha fin coincide con la fecha de grado del estudiante?','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}, # 1 -1
              'TES_PREG_P': {'TP_P_TABLE':['Número de cédula del graduado','¿La fecha fin coincide con la fecha de grado del estudiante?','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}, # 1 -1
              'ASE_PRG_ACA_P': {'APGA_P_TABLE':['Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'ASE_CRE_CUR_P': {'ACC_P_TABLE':['Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']},
              'ASE_PRG_ONDAS_P': {'APO_P_TABLE':['Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}},
    'NC' : {'LIB' : {'LIB_T_AVAL_TABLE': ['Proyecto de investigación del cual se derivó el libro (Código-Título)','Financiador(es) del proyecto del cual se derivó el libro', 'Financiador(es) de la publicación','Autores','Citas recibidas (si tiene)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}, 
            'CAP_LIB':{'CAP_LIB_T_AVAL_TABLE':['Proyecto de investigación del cual se derivó el libro que contiene el capítulo (Código-Título)','Financiador del proyecto del cual se derivó el libro que contiene el capítulo','Financiador de la publicación','Autores','Citas recibidas (si tiene)','Agregue las evidencias verificadas al repositorio digital y genere un hipervínculo en este campo','¿El producto cumple con los requisitos para ser avalado?']}}
}

In [90]:
d = {
                '1': 'C',
                '2': 'D',
                '3': 'E',
                '4': 'F',
                '5': 'G',
                '6': 'H',
                '7': 'I',
                '8': 'J',
                '9': 'K',
                '10': 'L',
                '11': 'M',
                '12': 'N',
                '13': 'O',
                '14': 'P',
                '15': 'Q',
                '16': 'R',
                '17': 'S',
                '18': 'T',
                '19': 'U',
                '20': 'V'
}

In [91]:
def clean_df(df):
    'remove innecesari collums'
    c=[x for x in df.columns if x.find('Unnamed:') == -1 and  x.find('Revisar') == -1]
    dfc=df[c]
    return dfc

In [92]:
1. Presentación: 
2. Datos de contacto: comprende información relativa al Grupo y al Centro, Instituto o Corporación que verifica la información. Diligenciar fila 8.
3. Integrantes: comprende la información relacionada a los integrantes de los grupos de investigación que se debe verificar y completar.
4. ART y N: 
5. LIB Y LIB_FOR: 
6. CAP:
7. Patente y variedades: 
8. AAD:   
9. Tecnológico: 
10. Empresarial: 
11. ASC y Divulgación Pública de la Ciencia: 
12. Formación y programas:

SyntaxError: invalid syntax (<ipython-input-92-3f5250ce3284>, line 1)

In [93]:
def format_df(df, sheet_name, start_row, writer,eh, veh =None):
    'format headers'
    
    df.to_excel(writer,sheet_name, startrow = start_row+1, startcol=2,index = False)

    # Get the xlsxwriter workbook and worksheet objects.
    worksheet = writer.sheets[sheet_name]
    
    #form merge cells
    start,end = 1,df.shape[1]

    m_range = d.get(str(start)) + str(start_row + 1) + ':' + d.get(str(end)) + str(start_row +1)

    worksheet.merge_range(m_range, 'Información suministrada por la Vicerrectoría de Investigación', merge_format)
    
    # for merge cells
    _m_range = d.get(str(end+1)) + str(start_row +1) + ':' +  d.get(str(end+len(eh))) + str(start_row +1)
    
    worksheet.merge_range(_m_range, 'Validación del Centro, Instituto o Corporación', merge_format)
        
    worksheet.set_row_pixels(start_row+1, 120)
    #worksheet.set_column('C:C',30,general)
    
    # SET COLUMS BY SHEET
    
 
    if sheet_name == '3.Integrantes grupo':
        
        worksheet.set_column('C:C', 10,general)
        worksheet.set_column('D:D', 30,general)
        worksheet.set_column('E:G', 15,general)
        
    elif sheet_name== 'NC_P':
        
        worksheet.set_column('C:C', 30,general)
        worksheet.set_column('D:K', 20,general)
    elif sheet_name == 'FRH_P':
        
        worksheet.set_column('C:C', 30,general)
        worksheet.set_column('D:K', 20,general)
    else:
        worksheet.set_column('C:C', 30,general)
        worksheet.set_column('D:K', 20,general)
    
    worksheet.set_column('A:A', 15)
    worksheet.set_column('B:B', 5)
        
    worksheet.write(start_row+1, 0, 'VoBo de VRI', merge_format)
    # Add a header format.
    
    fmt_header = workbook.add_format({
        'bold': True,
        'align': 'center',    
        'text_wrap': True,
        'valign': 'vcenter',
        'fg_color': '#33A584',
        'font_color': '#FFFFFF',
        'border': 1})
    
    
    # Write the column headers with the defined format.
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(start_row+1, col_num + 2, value, fmt_header)
        
    # write extra headers
    for col_num, value in enumerate(eh):
        worksheet.write(start_row+1, col_num + df.shape[1] + 2, value, fmt_header)

In [94]:
def format_info(df, writer, sheet_name):
    
    '''format worksheet'''
    
    workbook=writer.book
    
    normal=workbook.add_format({'font_size':12,'text_wrap':True})
    
    merge_format = workbook.add_format({
    'bold': 1,
    'border':1,
    'text_wrap': True,    
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black'})
    
    fmt_header = workbook.add_format({
        'align': 'center',    
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#33A584',
        'font_color': '#FFFFFF',
        'border': 1})
    
    # write df
    start_row = 6
    start_col = 3
    
    df.to_excel(writer, sheet_name, startrow =start_row, startcol=start_col,index = False)

    # get worksheet object
    worksheet = writer.sheets[sheet_name]
    
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(start_row, col_num + 3, value, fmt_header)
    
    #Prepare image insertion: See → https://xlsxwriter.readthedocs.io/example_images.html
    worksheet.set_column('A:A', 15)
    worksheet.set_column('B:B', 15)
    worksheet.insert_image('A1', 'img/udea.jpeg')
    
    # title 1 UNIVERSIDAD DE ANTIOQUIA
    title = workbook.add_format({'font_size':16,'center_across':True})

    # title 2 Vicerrectoria de Investigación
    title2 = workbook.add_format({'font_size':16,'center_across':True})
   
    # sub title 2 datos identificacion contacto
    title3 = workbook.add_format({'font_size':12,'center_across':True})
    
    # merge d1:f1
    worksheet.merge_range('D1:F1', 'UNIVERSIDAD DE ANTIOQUIA', title)
        
    # merge d2:f2
    worksheet.merge_range('D2:F2', ' Vicerrectoria de Investigación', title2)
    
    # merge d3:f3
    worksheet.merge_range('D3:F3', ' Datos de identificación y contacto', title3)
    
    # D5: F5
    worksheet.merge_range('D5:E5','Número inscripcion a la convocatoria:',merge_format)
    worksheet.write('F5','#',merge_format)
    
    # d6:f6
    worksheet.merge_range('D6:F6','Identificación del Grupo',merge_format)
        
    # d9:f9
    worksheet.merge_range('D10:F10','Identificación del Centro de Investigación',merge_format)
    # write 
    a='Nombre del Centro, Instituto o Corporación'
    worksheet.write('D11',a, fmt_header)
    worksheet.set_column('D11:D11',30, fmt_header)
    
    b='Nombre completo del Jefe de Centro, Instituto o Corporación'
    worksheet.write('E11',b, fmt_header) 
    worksheet.set_column('E11:E11',30, fmt_header)
    
    c='Email'
    worksheet.write('F11',c, fmt_header) 
    worksheet.set_column('F11:F11',30, fmt_header)
    
    # d13:f13
    worksheet.merge_range('D13:F13','Identificación de quien diligencia el formato',merge_format)
    a='Nombre completo del encargado de diligenciar el formato'
    worksheet.write('D14',a, fmt_header)
    worksheet.set_column('D14:D14',30, normal)
    
    b='Email'
    worksheet.write('E14',b, fmt_header) 
    worksheet.set_column('E14:E14',30, normal)
    
    c='Teléfono de contacto'
    worksheet.write('F14',c, fmt_header) 
    worksheet.set_column('F14:F14',30, normal)

In [95]:
# ONE GROUP IMPLEMENTATION
idxx = 34

# DATA
DBG = DB[0]

### excel name
name = 'AA_Plantilla_Formato de verificación de información_GrupLAC_894-2021_%s'

# initialize object= output excel file
writer = pd.ExcelWriter(name % dfg.loc[idxx,'COL Grupo']+'.xlsx', engine='xlsxwriter')

#Global variables
abstract_text='VERIFICACIÓN DE INFORMACIÓN PARA OTORGAR AVAL A LOS GRUPOS DE INVESTIGACIÓN  E INVESTIGADORES PARA SU PARTICIPACIÓN EN LA CONVOCATORIA 894 DE 2021 DE MINCIENCIAS'
instructions='''Los grupos de investigación e investigadores de la Universidad de Antioquia que deseen participar en la Convocatoria Nacional para el reconocimiento y medición de grupos de investigación, desarrollo tecnológico o de innovación y para el reconocimiento de investigadores del Sistema Nacional de Ciencia, Tecnología e Innovación - SNCTI, 894 de 2021, deben presentar la información actualizada en las plataformas CvLAC y GrupLAC validada por el Centro de Investigación en el presente formato, y respaldada en el repositorio digital de evidencias dispuesto para este fin, para la obtención del aval institucional por parte de la Vicerrectoría de Investigación. 

La información a validar corresponde a los años 2019-2020 y aquella que entra en la ventana de observación y debe ser modificada según el Modelo de medición de grupos. La validación comprende:

1. Verificación de la vinculación de los integrantes a la Universidad de Antioquia y al grupo de investigación.  Diligenciar los campos solicitados. 

2. Verificación de la producción de GNC, DTeI, ASC y FRH, en los campos habilitados en cada hoja de este formato. Las evidencias requeridas para los productos deben ser anexadas al repositorio digital asignado al grupo y se deben enlazar a cada producto.  

Este documento debe ser diligenciado en línea.

De antemano, la Vicerrectoría de Investigación agradece su participación en este ejercicio, que resulta de vital importancia para llevar a buen término la Convocatoria de Reconocimiento y Medición de Grupos de Investigación
'''
#Final part of the first sheet
datos=clean_df(pd.read_excel('https://github.com/restrepo/InstituLAC/raw/main/data/template_data.xlsx'))

#Capture xlsxwriter object 
# IMPORTANT → workbook is the same object used in the official document at https://xlsxwriter.readthedocs.io
workbook=writer.book
#***************
#Styles as explained in https://xlsxwriter.readthedocs.io
general=workbook.add_format({'text_wrap':True})
title=workbook.add_format({'font_size':28,'center_across':True})
subtitle=workbook.add_format({'font_size':24,'center_across':True})
abstract=workbook.add_format({'font_size':20,'center_across':True,'text_wrap':True})
normal=workbook.add_format({'font_size':12,'text_wrap':True})

merge_format = workbook.add_format({
    'bold': 1,
    'border':1,
    'text_wrap': True,    
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'blue'})

fmt_header = workbook.add_format({
    'bold': True,
    'align': 'center',    
    'text_wrap': True,
    'valign': 'top',
    'fg_color': '#33A584',
    'font_color': '#FFFFFF',
    'border': 1})
#***************
#Creates the first work-sheet
#IMPORTANT → worksheet is the same object  used in the official document at https://xlsxwriter.readthedocs.io
worksheet=workbook.add_worksheet("1.Presentación")
#Prepare image insertion: See → https://xlsxwriter.readthedocs.io/example_images.html
worksheet.set_column('A:A', 15)
worksheet.set_column('B:B', 15)
worksheet.insert_image('A1', 'img/udea.jpeg')
#Prepare text insertion: See  → https://xlsxwriter.readthedocs.io/example_images.html
worksheet.set_column('C:C', 140,general)
worksheet.set_row_pixels(0, 60)
#Texts
worksheet.write('C1', 'UNIVERSIDAD DE ANTIOQUIA',title)
worksheet.set_row_pixels(2, 60)
worksheet.write('C3', 'VICERRECTORÍA DE INVESTIGACIÓN',subtitle)
worksheet.set_row_pixels(5, 100)
worksheet.write('C6', abstract_text,abstract)
worksheet.set_row_pixels(8, 40)
worksheet.write('C9','PRESENTACIÓN DEL EJERCICIO',
                workbook.add_format({'font_size':18,'center_across':True}) )
worksheet.set_row_pixels(10, 320)
worksheet.write('C11',instructions,normal )
#*** ADD PANDAS DATAFRAME IN SPECIFIC POSITION ****
#Add a data Frame in some specific position. See → https://stackoverflow.com/a/43510881/2268280
#                                       See also → https://xlsxwriter.readthedocs.io/working_with_pandas.html
writer.sheets["1.Presentación"]=worksheet
datos.to_excel(writer,sheet_name="1.Presentación",startrow=12,startcol=2,index=False)
#**************************************************
#Fix columns heights for long text
worksheet.set_row_pixels(17, 40)
worksheet.set_row_pixels(18, 40)
worksheet.set_row_pixels(19, 40)
worksheet.set_row_pixels(20, 40)
worksheet.set_row_pixels(22, 40)

In [96]:
''' # Add extra headers
worksheet2.write(1, 0, 'VoBo de VRI', merge_format)
#New columns
extra_url='Si no tiene URL o DOI agregue una evidencia en el repositorio digital y genere un hipervínculo en este campo'
cols=['DOI',extra_url,'¿El producto cumple con los requisitos para ser avalado?']
for col , value in enumerate(cols):
    worksheet2.write(1, col+2+table.columns.size, value, fmt_header)
worksheet2.set_column('L:L',20)
worksheet2.set_column('M:M',20)
#Creates a set of cells with a drop-down menu Sí/No. See → https://xlsxwriter.readthedocs.io/working_with_data_validation.html
worksheet2.data_validation('M3:M{}'.format(table.shape[0]+2), {'validate': 'list',
                                  'source': ['Sí', 'No']})

workbook.close()
'''

" # Add extra headers\nworksheet2.write(1, 0, 'VoBo de VRI', merge_format)\n#New columns\nextra_url='Si no tiene URL o DOI agregue una evidencia en el repositorio digital y genere un hipervínculo en este campo'\ncols=['DOI',extra_url,'¿El producto cumple con los requisitos para ser avalado?']\nfor col , value in enumerate(cols):\n    worksheet2.write(1, col+2+table.columns.size, value, fmt_header)\nworksheet2.set_column('L:L',20)\nworksheet2.set_column('M:M',20)\n#Creates a set of cells with a drop-down menu Sí/No. See → https://xlsxwriter.readthedocs.io/working_with_data_validation.html\nworksheet2.data_validation('M3:M{}'.format(table.shape[0]+2), {'validate': 'list',\n                                  'source': ['Sí', 'No']})\n\nworkbook.close()\n"

In [97]:
DBEH.keys()

dict_keys(['INFO_GROUP', 'MEMBERS', 'NC_P', 'DTI_P', 'ASC_P', 'FRH_P', 'NC'])

In [98]:
# INFO GROUP
df=get_info(DBG['Info_group'])
format_info(df, writer, '2.Datos de contacto')

# WORKSHEET 1
df = clean_df(DBG['Members']) 
eh = DBEH['MEMBERS']
format_df(df, '3.Integrantes grupo', 1, writer, eh, veh=0) #### veh = 0

In [101]:
### NC_P ### 

#------- w4 -------
# 4.ART y N

var_w4 = 0

try:
    df=clean_df(DBG['NC_P']['ART_IMP_P']['ART_P_TABLE'])

    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc+1)
    
    eh=DBEH['NC_P']['ART_IMP_P']['ART_P_TABLE']

    format_df(df, '4.ART y N',  var_w4, writer,eh)
        
    var_w4 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['NC_P']['ART_ELE_P']['ART_E_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['ART_ELE_P']['ART_E_P_TABLE']

    format_df(df, '4.ART y N', var_w4, writer,eh)
        
    var_w4 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['NC_P']['NOT_CIE_P']['NOT_CIE_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['NOT_CIE_P']['NOT_CIE_P_TABLE']

    format_df(df, '4.ART y N', var_w4, writer,eh)
        
    var_w4 += df.shape[0] + 3
    
except KeyError as e:
    
    pass
# -------------- w4 -------------------------

#------------ ---w5------------
# 5.LIB y LIB_FOR
var_w5 = 0

# libros por pertenencia
try:
    df=clean_df(DBG['NC_P']['LIB_P']['LIB_P_TABLE'])

    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['LIB_P']['LIB_P_TABLE']

    format_df(df, '5.LIB y LIB_FOR',  var_w5, writer,eh)
        
    var_w5 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# libros avalados con revisión
try:
    df=clean_df(DBG['NC']['LIB']['LIB_T_AVAL_TABLE'])  ### ,veh = 2

    #df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    eh=DBEH['NC']['LIB']['LIB_T_AVAL_TABLE']

    format_df(df, '5.LIB y LIB_FOR', var_w5 , writer, eh)
        
    var_w5  += df.shape[0] + 3

except KeyError as e:
    
    pass

# libros formacion
try:
    df=clean_df(DBG['ASC_P']['GEN_CONT_IMP_P']['GC_I_P_TABLE_5']) # lib form

    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['GEN_CONT_IMP_P']['GC_I_P_TABLE_5']

    format_df(df, '5.LIB y LIB_FOR',  var_w5 , writer,eh)
        
    var_w5 += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass  
# --------------------w5--------------

#--------------------w6---------------
#6.CAP

# cap pertenencia

var_w6 = 0

try:
    df=clean_df(DBG['NC_P']['CAP_LIB_P']['CAP_LIB_P_TABLE'])

    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['CAP_LIB_P']['CAP_LIB_P_TABLE']

    format_df(df, '6.CAP',var_w6, writer,eh)
        
    var_w6 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# caps avalados con revision
try:
    df = clean_df(DBG['NC']['CAP_LIB']['CAP_LIB_T_AVAL_TABLE'])  ### ,veh = 2

    #df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    eh = DBEH['NC']['CAP_LIB']['CAP_LIB_T_AVAL_TABLE']

    format_df(df, '6.CAP', var_w6, writer, eh)
        
    var_w6 += df.shape[0] + 3

except KeyError as e:
    
    pass

# traduccion filologica
try:
    df=clean_df(DBG['NC_P']['TRA_FIL_P']['TRA_FIL_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['TRA_FIL_P']['TRA_FIL_P_TABLE']

    format_df(df, '6.CAP', var_w6, writer,eh)
        
    var_w6 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

#-------------------w6------------------

#------------w7-------------------------
#7.Patente_Variedades
var_w7 = 0

# patentes
try:
    df=clean_df(DBG['NC_P']['PAT_P']['PAT_P_TABLE']) ###### veh=1
    
    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['PAT_P']['PAT_P_TABLE']

    format_df(df, '7.Patente_Variedades', var_w7, writer,eh, veh=1)
        
    var_w7 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# productos investigacion creacion
try:
    df=clean_df(DBG['NC_P']['PRD_INV_ART_P']['PAAD_P_TABLE']) ###### veh = 1

    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['PRD_INV_ART_P']['PAAD_P_TABLE']

    format_df(df, '7.Patente_Variedades', var_w7, writer,eh, veh=1)
        
    var_w7 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# variedad vegetal
try:
    df=clean_df(DBG['NC_P']['VAR_VEG_P']['VV_P_TABLE'])

    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['VAR_VEG_P']['VV_P_TABLE']

    format_df(df, '7.Patente_Variedades', var_w7, writer,eh)
        
    var_w7 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# Variedad Animal
try:
    df=clean_df(DBG['NC_P']['VAR_ANI_P']['VA_P_TABLE'])

    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['VAR_ANI_P']['VA_P_TABLE']

    format_df(df, '7.Patente_Variedades', var_w7, writer,eh)
        
    var_w7 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# razas pecuarias mejoradas
try:
    df=clean_df(DBG['NC_P']['RAZ_PEC_P']['RAZ_PEC_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='NC_P',startrow = var_nc)
    
    eh=DBEH['NC_P']['RAZ_PEC_P']['RAZ_PEC_P_TABLE']

    format_df(df, '7.Patente_Variedades', var_w7, writer,eh)
        
    var_w7 += df.shape[0] + 3

except KeyError as e:
    
    pass
# ---------------w7---------------------

#---------------w8-------------------
var_w8 = 0

# registros de acuerdo licencias expl obras AAD
try:

    df=clean_df(DBG['DTI_P']['REG_AAD_P']['AAAD_P_TABLE'])
    
    eh=DBEH['DTI_P']['REG_AAD_P']['AAAD_P_TABLE']

    format_df(df, '8.AAD', var_w8, writer,eh)
        
    var_w8 += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

#-------------W8---------------------

#-------------W9----------------

# 9.Tecnológico
#### DTI_P

var_w9 = 0

# diseño industrial
try:

    df=clean_df(DBG['DTI_P']['DIS_IND_P']['DI_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['DIS_IND_P']['DI_P_TABLE']

    format_df(df, '9.Tecnológico', var_w9, writer, eh)
        
    var_w9 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

#circuitos integrados
try:
    df=clean_df(DBG['DTI_P']['CIR_INT_P']['ECI_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['CIR_INT_P']['ECI_P_TABLE']

    format_df(df, '9.Tecnológico', var_w9, writer,eh)
        
    var_w9 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# colecciones
try:
    df=clean_df(DBG['DTI_P']['COL_CIENT_P']['COL_CIENT_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['COL_CIENT_P']['COL_CIENT_P_TABLE']

    format_df(df, '9.Tecnológico', var_w9, writer,eh)
        
    var_w9 += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

# software 
try:
    df=clean_df(DBG['DTI_P']['SOFT_P']['SF_P_TABLE'])
    
    eh=DBEH['DTI_P']['SOFT_P']['SF_P_TABLE']

    format_df(df, '9.Tecnológico', var_w9, writer,eh)
        
    var_w9 += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

# secreto industrial
try:
    df=clean_df(DBG['DTI_P']['SEC_IND_P']['SE_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['SEC_IND_P']['SE_P_TABLE']

    format_df(df, '9.Tecnológico', var_w9, writer,eh)
        
    var_w9 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# prototipo insdustrial
try:
    df=clean_df(DBG['DTI_P']['PRT_IND_P']['PI_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['PRT_IND_P']['PI_P_TABLE']

    format_df(df, '9.Tecnológico',  var_w9, writer,eh)
        
    var_w9 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# Registro distintivo
try:
    df=clean_df(DBG['DTI_P']['SIG_DIS_P']['SD_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['SIG_DIS_P']['SD_P_TABLE']

    format_df(df, '9.Tecnológico', var_w9, writer,eh)
        
    var_w9 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# prod nutracetico
try:
    df=clean_df(DBG['DTI_P']['NUTRA_P']['NUTRA_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_nc)
    
    eh=DBEH['DTI_P']['NUTRA_P']['NUTRA_P_TABLE']

    format_df(df, '9.Tecnológico', var_w9, writer,eh)
        
    var_w9 += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

# registro cienti
try:
    df=clean_df(DBG['DTI_P']['REG_CIENT_P']['REG_CIENT_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['REG_CIENT_P']['REG_CIENT_P_TABLE']

    format_df(df, '9.Tecnológico',var_w9 , writer,eh)
        
    var_w9 += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

# planta piloto

try:
    df=clean_df(DBG['DTI_P']['PLT_PIL_P']['PP_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['PLT_PIL_P']['PP_P_TABLE']

    format_df(df, '9.Tecnológico', var_w9, writer,eh)
        
    var_w9 += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

# prototipo vigilancia epidemologica

try:
    df=clean_df(DBG['DTI_P']['PROT_VIG_EPID_P']['PROT_VIG_EPID_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['PROT_VIG_EPID_P']['PROT_VIG_EPID_P_TABLE']

    format_df(df, '9.Tecnológico',var_w9, writer,eh)
        
    var_w9 += df.shape[0] + 3
    
except KeyError as e:
    
    pass
#---------------------w9----------------

#---------------------w10----------------
# 10.Empresarial
var_w10 = 0

# innovación gestion empresarial
try:
    df=clean_df(DBG['DTI_P']['INN_GES_EMP_P']['IG_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['INN_GES_EMP_P']['IG_P_TABLE']

    format_df(df, '10.Empresarial', var_w10, writer,eh)
        
    var_w10 += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass


# innovacion procesos y procedimiento
try:
    df=clean_df(DBG['DTI_P']['INN_PROC_P']['IPP_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['INN_PROC_P']['IPP_P_TABLE']

    format_df(df, '10.Empresarial', var_w10, writer,eh)
        
    var_w10 += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

# regulaciones normas reglamentos legislaciones
try:
    df=clean_df(DBG['DTI_P']['REG_NORM_REGL_LEG_P']['RNR_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['REG_NORM_REGL_LEG_P']['RNR_P_TABLE']

    format_df(df, '10.Empresarial', var_w10, writer,eh)
        
    var_w10 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# conceptos tecnicos
try:
    df=clean_df(DBG['DTI_P']['CONP_TEC_P']['CONP_TEC_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['CONP_TEC_P']['CONP_TEC_P_TABLE']

    format_df(df, '10.Empresarial', var_w10, writer,eh)
        
    var_w10 += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

# empresa base tecnologica
try:
    df=clean_df(DBG['DTI_P']['EMP_BSE_TEC_P']['EBT_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['EMP_BSE_TEC_P']['EBT_P_TABLE']

    format_df(df, '10.Empresarial', var_w10, writer,eh)
        
    var_w10 += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

# empresa de base cultural
try:
    df=clean_df(DBG['DTI_P']['EMP_CRE_CUL_P']['ICC_P_TABLE'])

    #df.to_excel(writer,sheet_name='DTI_P',startrow = var_dt)
    
    eh=DBEH['DTI_P']['EMP_CRE_CUL_P']['ICC_P_TABLE']

    format_df(df, '10.Empresarial', var_w10, writer,eh)
        
    var_w10 += df.shape[0] + 3
    
    
except KeyError as e:
    
    pass

# -------------------------w10-------------
######  ASC

# -------- w11
# 11.ASC y Divulgación
var_w11 = 0 

# productos de interes social
try:
    df=clean_df(DBG['ASC_P']['PASC_P']['PASC_FOR_P_TABLE'])
    
    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['GEN_CONT_IMP_P']['GC_I_P_TABLE_5']

    format_df(df, '11.ASC y Divulgación', var_w11, writer,eh)
        
    var_w11 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# Proceso de apropiación social del conocimiento resultado del trabajo conjunto 
try:
    df=clean_df(DBG['ASC_P']['PASC_P']['PASC_TRA_P_TABLE'])

    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['PASC_P']['PASC_TRA_P_TABLE']

    format_df(df, '11.ASC y Divulgación', var_w11, writer,eh)
        
    var_w11 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['ASC_P']['PASC_P']['PASC_GEN_P_TABLE'])

    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['PASC_P']['PASC_GEN_P_TABLE']

    format_df(df, '11.ASC y Divulgación', var_w11, writer,eh)
        
    var_w11 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

try:
    df=clean_df(DBG['ASC_P']['PASC_P']['PASC_CAD_P_TABLE'])

    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['PASC_P']['PASC_CAD_P_TABLE']

    format_df(df, '11.ASC y Divulgación', var_w11, writer,eh)
        
    var_w11 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# Divulgacion
# Piezas digitales
try:
    df=clean_df(DBG['ASC_P']['DC_P']['DC_CD_P_TABLE'])

    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['DC_P']['DC_CD_P_TABLE']

    format_df(df, '11.ASC y Divulgación', var_w11, writer,eh)
        
    var_w11 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# textuales
try:
    df=clean_df(DBG['ASC_P']['DC_P']['DC_CON_P_TABLE'])

    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['DC_P']['DC_CON_P_TABLE']

    format_df(df, '11.ASC y Divulgación', var_w11, writer,eh)
        
    var_w11 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# produccion estrategia trasmediatica
try:
    df=clean_df(DBG['ASC_P']['DC_P']['DC_TRA_P_TABLE'])

    #df.to_excel(writer,sheet_name='ASC_P',startrow = var_as)
    
    eh=DBEH['ASC_P']['DC_P']['DC_TRA_P_TABLE']

    format_df(df, '11.ASC y Divulgación', var_w11, writer,eh)
        
    var_w11 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# desarrollo web
try:
    df=clean_df(DBG['ASC_P']['DC_P']['DC_DES_P_TABLE'])

    eh=DBEH['ASC_P']['DC_P']['DC_DES_P_TABLE']

    format_df(df, '11.ASC y Divulgación', var_w11, writer,eh)
        
    var_w11 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# --- --- --- -- w11 -- -- -- -- -- -- --

# ---------------w12--------------------

# FRH

var_w12 = 0

# tesis doctorado
try:
    df=clean_df(DBG['FRH_P']['TES_DOC_P']['TD_P_TABLE'])  ### ,veh = 2

    #df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    eh=DBEH['FRH_P']['TES_DOC_P']['TD_P_TABLE']

    format_df(df, '12.Formación y programas', var_w12, writer, eh)
        
    var_w12 += df.shape[0] + 3

except KeyError as e:
    
    pass

# tesis maestria
try:
    df=clean_df(DBG['FRH_P']['TES_MAST_P']['TM_P_TABLE']) ### veh = 2

    #df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    eh=DBEH['FRH_P']['TES_MAST_P']['TM_P_TABLE']

    format_df(df, '12.Formación y programas',var_w12, writer,eh)
        
    var_w12 += df.shape[0] + 3
    
except KeyError as e:
    
    pass
# tesis pregrado
try:
    df=clean_df(DBG['FRH_P']['TES_PREG_P']['TP_P_TABLE']) ### veh = 2

    #df.to_excel(writer,sheet_name='FRH_P',startrow = var_rh)
    
    eh=DBEH['FRH_P']['TES_PREG_P']['TP_P_TABLE']

    format_df(df, '12.Formación y programas',var_w12, writer,eh,veh = 2)
        
    var_w12 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# asesoria programa academico
try:
    df=clean_df(DBG['FRH_P']['ASE_PRG_ACA_P']['APGA_P_TABLE']) 
    
    eh=DBEH['FRH_P']['ASE_PRG_ACA_P']['APGA_P_TABLE']

    format_df(df, '12.Formación y programas', var_w12, writer,eh)
        
    var_w12 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# asesoria creacion de cursos
try:
    df=clean_df(DBG['FRH_P']['ASE_CRE_CUR_P']['ACC_P_TABLE'])
    
    eh=DBEH['FRH_P']['ASE_CRE_CUR_P']['ACC_P_TABLE']

    format_df(df, '12.Formación y programas', var_w12, writer,eh)
        
    var_w12 += df.shape[0] + 3
    
except KeyError as e:
    
    pass

# programa ondas
try:
    df=clean_df(DBG['FRH_P']['ASE_PRG_ONDAS_P']['APO_P_TABLE'])

    eh=DBEH['FRH_P']['ASE_PRG_ONDAS_P']['APO_P_TABLE']

    format_df(df, '12.Formación y programas', var_w12, writer,eh)
        
    var_w12 += df.shape[0] + 3
    
except KeyError as e:
    
    pass
#----------------w12---------------------------
writer.save()

# ------

In [37]:
'''
schema for store tables

DBG = {
    
    'INFO_GROUP': 'TABLE',
    'MEMBERS':'TABLE',
       
    'NC_P': {'ART_IMP_P': {'ART_P_TABLE':'TABLE'},
             'ART_ELE_P': {'ART_E_P_TABLE':'TABLE'},
             'LIB_P':     {'LIB_P_TABLE':'TABLE'},
             'CAP_LIB_P': {'CAP_LIB_P_TABLE':'TABLE'},
             'NOT_CIE_P': {'NOT_CIE_P_TABLE':'TABLE'},
             'PAT_P':     {'PAT_P_TABLE':'TABLE'},
             'PRD_INV_ART_P': {'PAAD_P_TABLE':'TABLE'},
             'VAR_VEG_P':     {'VV_P_TABLE':'TABLE'},
             'VAR_ANI_P':     {'VA_P_TABLE':'TABLE'},
             'RAZ_PEC_P':     {'RAZ_PEC_P_TABLE':'TABLE'},
             'TRA_FIL_P': {'TRA_FIL_P_TABLE':'TABLE'}
            },
     'DTI_P': {'DIS_IND_P': {'DI_P_TABLE':'TABLE'},
              'CIR_INT_P': {'ECI_P_TABLE':'TABLE'},
              'SOFT_P': {'SF_P_TABLE':'TABLE'},
              'NUTRA_P': {'NUTRA_P_TABLE':'TABLE'},
              'COL_CIENT_P': {'COL_CIENT_P_TABLE':'TABLE'},
              'REG_CIENT_P': {'REG_CIENT_P_TABLE':'TABLE'},
              'PLT_PIL_P': {'PP_P_TABLE':'TABLE'},
              'PRT_IND_P': {'PI_P_TABLE':'TABLE'},
              'SEC_IND_P': {'SE_P_TABLE':'TABLE'},
              'PROT_VIG_EPID_P': {'PROT_VIG_EPID_P_TABLE':'TABLE'},
              'EMP_BSE_TEC_P': {'EBT_P_TABLE':'TABLE'},
              'EMP_CRE_CUL_P': {'ICC_P_TABLE':'TABLE'},
              'INN_GES_EMP_P': {'IG_P_TABLE':'TABLE'},
              'INN_PROC_P': {'IPP_P_TABLE':'TABLE'},
              'REG_NORM_REGL_LEG_P': {'RNR_P_TABLE':'TABLE'},
              'CONP_TEC_P': {'CONP_TEC_P_TABLE':'TABLE'},
              'REG_AAD_P': {'AAAD_P_TABLE':'TABLE'},
              'SIG_DIS_P': {'SD_P_TABLE':'TABLE'}
              },
    'ASC_P': {'GEN_CONT_IMP_P': {'GC_I_P_TABLE_5':'TABLE'},
              'PASC_P': {'PASC_FOR_P_TABLE':'TABLE',
               'PASC_TRA_P_TABLE':'TABLE',
               'PASC_GEN_P_TABLE':'TABLE',
               'PASC_CAD_P_TABLE':'TABLE'},
              'DC_P': {'DC_CD_P_TABLE':'TABLE',
               'DC_CON_P_TABLE':'TABLE',
               'DC_TRA_P_TABLE':'TABLE',
               'DC_DES_P_TABLE':'TABLE'}},
    'FRH_P': {'TES_DOC_P': {'TD_P_TABLE':'TABLE'},
              'TES_MAST_P': {'TM_P_TABLE':'TABLE'},
              'TES_PREG_P': {'TP_P_TABLE':'TABLE'},
              'PROY_INV_DES_P': {'PID_P_TABLE':'TABLE'},
              'PROY_INV_CRE_P': {'INV_CRE_P_TABLE':'TABLE'},
              'PROY_INV_DES_INN_P': {'PF_P_TABLE':'TABLE'},
              'PROY_INV_RESP_SOC_P': {'PE_P_TABLE':'TABLE'},
              'ASE_PRG_ACA_P': {'APGA_P_TABLE':'TABLE'},
              'ASE_CRE_CUR_P': {'ACC_P_TABLE':'TABLE'},
              'ASE_PRG_ONDAS_P': {'APO_P_TABLE':'TABLE'}}
}'''